# 📑 Tutorials for PyPOTS Clustering Models

## 📀 Preparing the **PhysioNet-2012** dataset for this tutorial

In [1]:
from pypots.data.generating import gene_physionet2012

# Load the PhysioNet-2012 dataset, disable artificially-missing values for evaluation
physionet2012_dataset = gene_physionet2012(artificially_missing=False)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())


2023-05-16 09:29:08 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Database)...
2023-05-16 09:29:08 [INFO]: Starting preprocessing physionet_2012...


Dataset physionet_2012 has already been downloaded. Processing directly...
Dataset physionet_2012 has already been cached. Loading from cache directly...
Loaded successfully!
dict_keys(['n_classes', 'n_steps', 'n_features', 'train_X', 'train_y', 'val_X', 'val_y', 'test_X', 'test_y', 'scaler'])


In [2]:
# Assemble the datasets for training, validating, and testing.
import numpy as np

# don't need validation set
dataset_for_training = {
    "X": np.concatenate([physionet2012_dataset['train_X'], physionet2012_dataset['val_X']], axis=0),
    "y": np.concatenate([physionet2012_dataset['train_y'], physionet2012_dataset['val_y']], axis=0),
}

dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "y": physionet2012_dataset['test_y'],
}


## 🚀 An exmaple of **CRLI** for clustering

In [7]:
from pypots.optim import Adam
from pypots.clustering import CRLI

# initialize the model
crli = CRLI(
    n_steps=physionet2012_dataset["n_steps"],
    n_features=physionet2012_dataset["n_features"],
    n_clusters=physionet2012_dataset["n_classes"],
    n_generator_layers=2,
    rnn_hidden_size=256,
    rnn_cell_type="GRU",
    decoder_fcn_output_dims=[256, 128],  # the output dimensions of layers in the decoder FCN.
    # Here means there are 3 layers. Leave it to default as None will results in
    # the FCN haveing only one layer.
    batch_size=32,
    epochs=10,  # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    # here we set patience=5 to early stop the training if the evaluting loss doesn't decrease for 5 epoches.
    patience=3,
                # You can leave it to defualt as None to disable early stopping.
    # give the generator optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    G_optimizer=Adam(lr=1e-3),
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    # give the discriminator optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    D_optimizer=Adam(lr=1e-3),
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    # this argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    num_workers=0,
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed//ssadfsadf
    device='cpu',  # just leave it to default, PyPOTS will automatically assign the best device for you.
                  # Set it to 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices.
    saving_path="tutorial_results/clustering/crli",  # set the path for saving tensorboard files
    model_saving_strategy="best",  # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
)


2023-05-16 08:40:03 [INFO]: the trained model will be saved to tutorial_results/clustering/crli/20230516_T084003, the tensorboard file will be saved to tutorial_results/clustering/crli/20230516_T084003/tensorboard
2023-05-16 08:40:03 [INFO]: Model initialized successfully with the number of trainable parameters: 1,546,820


In [8]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
crli.fit(train_set=dataset_for_training)


2023-05-16 08:42:54 [INFO]: epoch 0: training loss_generator 3.3691, train loss_discriminator 0.3934
2023-05-16 08:45:22 [INFO]: epoch 1: training loss_generator 3.4669, train loss_discriminator 0.3709
2023-05-16 08:48:00 [INFO]: epoch 2: training loss_generator 3.4688, train loss_discriminator 0.3631
2023-05-16 08:50:24 [INFO]: epoch 3: training loss_generator 3.4712, train loss_discriminator 0.3591
2023-05-16 08:50:24 [INFO]: Exceeded the training patience. Terminating the training procedure...
2023-05-16 08:50:24 [INFO]: Finished training.
2023-05-16 08:50:24 [INFO]: Saved the model to tutorial_results/clustering/crli/20230516_T084003/CRLI.pypots.


In [9]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
crli_prediction = crli.cluster(dataset_for_testing)


/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [10]:
from pypots.utils.metrics import cal_rand_index, cal_cluster_purity

# calculate mean absolute error on the ground truth (artificially-missing values)
RI = cal_rand_index(crli_prediction, dataset_for_testing["y"])
CP = cal_cluster_purity(crli_prediction, dataset_for_testing["y"])

print("Testing clustering metrics: \n"
      f'RI: {RI}, \n'
      f'CP: {CP}\n'
      )


Testing clustering metrics: 
RI: 0.7511829319593613, 
CP: 0.8548790658882403,



## 🚀 An exmaple of **VaDER** for clustering

In [3]:
from pypots.optim import Adam
from pypots.clustering import VaDER

# initialize the model
vader = VaDER(
    n_steps=physionet2012_dataset["n_steps"],
    n_features=physionet2012_dataset["n_features"],
    n_clusters=physionet2012_dataset["n_classes"],
    rnn_hidden_size=128,
    d_mu_stddev=2,
    pretrain_epochs=20,
    batch_size=32,
    epochs=10,  # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    # here we set patience=5 to early stop the training if the evaluting loss doesn't decrease for 5 epoches.
    patience=3,
                # You can leave it to defualt as None to disable early stopping.
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    optimizer=Adam(lr=1e-3),
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    # this argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    num_workers=0,
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    device='cpu',  # just leave it to default, PyPOTS will automatically assign the best device for you.
    # Set it to 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices.
    saving_path="tutorial_results/clustering/vader",  # set the path for saving tensorboard files
    model_saving_strategy="best",  # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
)


2023-05-16 09:29:35 [INFO]: the trained model will be saved to tutorial_results/clustering/vader/20230516_T092935, the tensorboard file will be saved to tutorial_results/clustering/vader/20230516_T092935/tensorboard
2023-05-16 09:29:35 [INFO]: Model initialized successfully with the number of trainable parameters: 293,642


In [4]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
vader.fit(train_set=dataset_for_training)


2023-05-16 09:41:52 [INFO]: epoch 0: training loss 0.8087
2023-05-16 09:42:56 [INFO]: epoch 1: training loss 0.6093
2023-05-16 09:44:01 [INFO]: epoch 2: training loss 0.6157
2023-05-16 09:45:06 [INFO]: epoch 3: training loss 0.6193
2023-05-16 09:46:11 [INFO]: epoch 4: training loss 0.6308
2023-05-16 09:46:11 [INFO]: Exceeded the training patience. Terminating the training procedure...
2023-05-16 09:46:11 [INFO]: Finished training.
2023-05-16 09:46:11 [INFO]: Saved the model to tutorial_results/clustering/vader/20230516_T092935/VaDER.pypots.


In [5]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
vader_prediction = vader.cluster(dataset_for_testing)


In [6]:
from pypots.utils.metrics import cal_rand_index, cal_cluster_purity

# calculate mean absolute error on the ground truth (artificially-missing values)
RI = cal_rand_index(vader_prediction, dataset_for_testing["y"])
CP = cal_cluster_purity(vader_prediction, dataset_for_testing["y"])

print("Testing clustering metrics: \n"
      f'RI: {RI}, \n'
      f'CP: {CP},\n'
      )


Testing clustering metrics: 
RI: 0.7517747893791342, 
CP: 0.8548790658882403,

